In [ ]:

import os
import time
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.environ.get("PINECONE_INDEX_NAME")

print(f"Verificando o índice '{PINECONE_INDEX_NAME}' no Pinecone...")
pc = Pinecone(api_key=PINECONE_API_KEY)

# Verifica se o índice já existe
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    print(f"Índice '{PINECONE_INDEX_NAME}' não encontrado. Criando um novo...")

    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=384,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

    print("Aguardando o índice ficar pronto...")
    time.sleep(10)

    print("Carregando documentos do arquivo 'info_turismo.txt'...")
    
    loader = TextLoader("data/info_turismo.txt", encoding="utf-8")
    documents = loader.load()

    print("Dividindo o texto em chunks...")
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    print(f"Inicializando modelo de embeddings (384 dimensões)...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    print(f"Adicionando {len(docs)} documentos ao índice '{PINECONE_INDEX_NAME}'...")
    PineconeVectorStore.from_documents(docs, embeddings, index_name=PINECONE_INDEX_NAME)

    print("\n✅ Índice criado e populado com sucesso!")

else:
    print(f"✅ Índice '{PINECONE_INDEX_NAME}' já existe e está pronto para ser usado.")

c:\ananconda3\envs\turismo_ia\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


Verificando o índice 'turismo' no Pinecone...
✅ Índice 'turismo' já existe e está pronto para ser usado.
